In [2]:
import json
import random

In [2]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\first_epoch_entities.txt', 'r', encoding='utf-8') as f:
    entity_list = eval(f.read())

In [3]:
entity_dict = dict()
for i in entity_list:
    entity_dict[i[0]] = i[1]

In [4]:
len(entity_dict)

1449

In [ ]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\first_epoch_trainset.json', 'r', encoding='utf-8') as f:
    first_epoch_trainset = json.loads(f.read())

In [5]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\test_set.json', 'r', encoding='utf-8') as f:
    test_set = json.loads(f.read())
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\validset.json', 'r', encoding='utf-8') as f:
    validset = json.loads(f.read())

In [6]:
def update_label(dataset: dict, entity_dict: dict):
    """更新数据集标签

    Args:
        dataset (dict): 数据集
        entity_dict (dict): 实体词典
    """
    # 代码有问题
    # 会把已经标注的标签会被多次标注，这不符合实体识别的原则，一个实体只能被标注一次
    for i, labels in enumerate(dataset['labels']):
        text = dataset['corpus'][i]
        existing_spans = {label['span'] for label in labels['output']}

        # 更新已有的标签
        for label in labels['output']:
            span = label['span']
            if span in entity_dict:
                # 更新type
                label['type'] = entity_dict[span]
                # 更新start和end
                start = text.find(span)
                if start != -1:
                    label['start'] = start
                    label['end'] = start + len(span)
                # 更新span
                label['span'] = span

        # 按照长度排序实体词典，确保最长的实体优先匹配
        sorted_entities = sorted(entity_dict.items(), key=lambda item: len(item[0]), reverse=True)


        # 添加未识别出的实体标签
        for span, entity_type in sorted_entities:
            start = text.find(span)
            if start != -1:
                overlap = False
                for label in labels['output']:
                    if (start >= label['start'] and start < label['end']) or (start + len(span) > label['start'] and start + len(span) <= label['end']):
                        overlap = True
                        break
                if not overlap:
                    new_label = {
                        'type': entity_type,
                        'start': start,
                        'end': start + len(span),
                        'prob': 0.0,  # 可以根据需要设置默认值
                        'span': span
                    }
                    labels['output'].append(new_label)

In [ ]:
update_label(test_set, entity_dict)
update_label(validset, entity_dict)

In [8]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\test_set_v2.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(test_set))

In [ ]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\validset_v2.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(validset))

In [ ]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\first_epoch_trainset_v3.json', 'r', encoding='utf-8') as f:
    first_epoch_trainset_v3 = json.loads(f.read())

In [ ]:
entity_set = set()
for i, row in enumerate(first_epoch_trainset_v3):
    if row.get('label'):
        entities = [label['text'] for label in row['label']]
        entities_type = [label['labels'][0] for label in row['label']]
        for item in dict(zip(entities, entities_type)).items():
            entity_set.add(item)
    else:
        continue

In [ ]:
entity_list = list(entity_set)
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\first_epoch_entities.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(entity_list))
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\first_epoch_entities.txt', 'w', encoding='utf-8') as f:
    f.write(str(entity_list))

In [ ]:
# validset 标记完之后，更新词典
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\first_epoch_entities.txt', 'r', encoding='utf-8') as f:
    first_epoch_entities = set(eval(f.read()))
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\validset_v3.json', 'r', encoding='utf-8') as f:
    validset_v3 = json.loads(f.read())

for i, row in enumerate(validset_v3):
    if row.get('label'):
        entities = [label['text'] for label in row['label']]
        entities_type = [label['labels'][0] for label in row['label']]
        for item in dict(zip(entities, entities_type)).items():
            first_epoch_entities.add(item)
    else:
        continue

with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\first_epoch_entities.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(list(first_epoch_entities)))
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\first_epoch_entities.txt', 'w', encoding='utf-8') as f:
    f.write(str(first_epoch_entities))

Adaseq数据集格式
```json
{
    "text": "鲁迅文学院组织有关专家",
    "labels": ["B-ORG", "I-ORG", "I-ORG", "I-ORG", "I-ORG", "O", "O", "O", "O", "O", "O"]
}
```

In [7]:
def process_labels(dataset: list):
    new_dataset = []
    for i, row in enumerate(dataset):
        tmp = ['O'] * len(row['text'])
        if row.get('label'):
            labels = row['label']
            for label in labels:
                for idx in range(label['start'], label['end']):
                    tmp[idx] = f'I-{label['labels'][0]}' if idx != label['start'] else f'B-{label['labels'][0]}'
        new_dataset.append(
                {
                    'text': row['text'],
                    'labels': tmp
                }
        )

    return new_dataset

In [ ]:
first_epoch_trainset_v4 = process_labels(first_epoch_trainset_v3)

In [ ]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\first_epoch_trainset_v4.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(first_epoch_trainset_v4))

In [ ]:
validset_v4 = process_labels(validset_v3)
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\validset_v4.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(validset_v4))

In [ ]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\first_epoch_trainset_v4.json', 'r', encoding='utf-8') as f:
    first_epoch_trainset_v4 = json.loads(f.read())
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\validset_v4.json', 'r', encoding='utf-8') as f:
    validset_v4 = json.loads(f.read())

In [8]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\test_set_v3.json', 'r', encoding='utf-8') as f:
    test_set_v3 = json.loads(f.read())
test_set_v4 = process_labels(test_set_v3)

In [10]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\test_set_v4.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(test_set_v4))

In [3]:
def json2conll(source, target):
    with open(source, "r", encoding="utf-8") as json_file:
        data_list = json.loads(json_file.read())

    conll_format = ""

    for data in data_list:
        text = data["text"]
        labels = data["labels"]

        for char, label in zip(text, labels):
            conll_format += f"{char}\t{label}\n"

        conll_format += "\n"  # Add a newline to separate sentences

    with open(target, 'w', encoding='utf-8') as conll:
        conll.write(conll_format)

In [16]:
json2conll(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\first_epoch_trainset_v4.json', r'G:\python_codes\Principles_of_Computer_Construction_KG\output\first_epoch_trainset_v4.txt')
json2conll(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\validset_v4.json', r'G:\python_codes\Principles_of_Computer_Construction_KG\output\validset_v4.txt')

In [11]:
json2conll(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\test_set_v4.json', r'G:\python_codes\Principles_of_Computer_Construction_KG\output\test_set_v4.txt')